마스크 데이터셋을 읽고 전처리를 진행한 후 데이터를 하나씩 꺼내주는 Dataset 클래스를 구현한 파일입니다.

이 곳에서, 나만의 Data Augmentation 기법 들을 구현하여 사용할 수 있습니다.

In [1]:
import os
import pandas as pd
from torch.utils.data import Dataset

In [2]:
get_data = pd.read_csv(r'/opt/ml/data/train/train.csv')
get_data = get_data.drop(['race','id'],axis=1)

In [3]:
get_data

,gender,age,path
0,female,45,000001_female_Asian_45
1,female,52,000002_female_Asian_52
2,male,54,000004_male_Asian_54
3,female,58,000005_female_Asian_58
4,female,59,000006_female_Asian_59
...,...,...,...
2695,male,19,006954_male_Asian_19
2696,male,19,006955_male_Asian_19
2697,male,19,006956_male_Asian_19
2698,male,20,006957_male_Asian_20


In [5]:
import torch

In [15]:
a = torch.tensor([1,2,3]).to('cuda')

In [16]:
torch.argmax(a).item()

2

In [17]:
import pandas as pd

In [18]:
target = pd.read_csv('submission.csv')
predict = pd.read_csv('/opt/ml/image-classification-level1-12/sm/guideline/results/08_28_12_42.csv')

a = torch.tensor(target['ans'])
b = torch.tensor(predict['ans'])

print(torch.sum(a==b))

tensor(8614)
